# Нейронные сети
__Суммарное количество баллов: 10__

Для начала вам предстоит реализовать свой собственный backpropagation и протестировать его на реальных данных, а затем научиться обучать нейронные сети при помощи библиотеки `PyTorch` и использовать это умение для классификации классического набора данных CIFAR10.

Обратите внимание, что использование PyTorch во всех заданиях кроме последнего запрещено. Автоматической проверки на его использование не будет, однако все посылки будут проверены вручную. 

In [1]:
import numpy as np
import copy
from sklearn.datasets import make_blobs, make_moons
from typing import List, NoReturn

### Задание 1 (3 балла)
Нейронные сети состоят из слоев, поэтому для начала понадобится реализовать их. Пока нам понадобятся только три:

`Linear` - полносвязный слой, в котором `y = Wx + b`, где `y` - выход, `x` - вход, `W` - матрица весов, а `b` - смещение. 

`ReLU` - слой, соответствующий функции активации `y = max(0, x)`.


#### Методы
`forward(X)` - возвращает предсказанные для `X`. `X` может быть как вектором, так и батчем

`backward(d)` - считает градиент при помощи обратного распространения ошибки. Возвращает новое значение `d`

`update(alpha)` - обновляет веса (если необходимо) с заданой скоростью обучения

#### Оценка
Валидируется корректность работы каждого модуля отдельно. Ожидается, что выходы каждого модуля будут незначительно отличаться от ожидаемых выходов, а подсчет градиента и градиентный спуск будут работать корректно.

In [2]:
class Module:
    """
    Абстрактный класс. Его менять не нужно. Он описывает общий интерфейс взаимодествия со слоями нейронной сети.
    """
    def forward(self, x):
        pass
    
    def backward(self, d):
        pass
        
    def update(self, alpha):
        pass
    
    
class Linear(Module):
    """
    Линейный полносвязный слой.
    """
    def __init__(self, in_features: int, out_features: int):
        """
        Parameters
        ----------
        in_features : int
            Размер входа.
        out_features : int 
            Размер выхода.
    
        Notes
        -----
        W и b инициализируются случайно.
        """
        self.in_features = in_features
        self.out_features = out_features
        self.W = np.random.normal(loc = 0.0, scale = np.sqrt(1/(self.in_features + self.out_features)), size = (self.out_features, self.in_features))
        self.b = np.zeros(self.out_features)
    
    def forward(self, x: np.ndarray) -> np.ndarray:
        """
        Возвращает y = Wx + b.

        Parameters
        ----------
        x : np.ndarray
            Входной вектор или батч.
            То есть, либо x вектор с in_features элементов,
            либо матрица размерности (batch_size, in_features).
    
        Return
        ------
        y : np.ndarray
            Выход после слоя.
            Либо вектор с out_features элементами,
            либо матрица размерности (batch_size, out_features)

        """
        self.x = x
        return np.dot(self.x, self.W.T) + self.b
    
    def backward(self, d: np.ndarray) -> np.ndarray:
        """
        Cчитает градиент при помощи обратного распространения ошибки.

        Parameters
        ----------
        d : np.ndarray
            Градиент.
        Return
        ------
        np.ndarray
            Новое значение градиента.
        """
        if len(self.x.shape) == 1:
            self.weights_grad = np.dot(d.reshape(len(d), 1), self.x.reshape(1, len(self.x)))
        else:   
            self.weights_grad = np.dot(d.T, self.x)
        self.b_grad = np.sum(d, axis = 0)
        return np.dot(d, self.W)
            
        
    def update(self, alpha: float) -> NoReturn:
        """
        Обновляет W и b с заданной скоростью обучения.

        Parameters
        ----------
        alpha : float
            Скорость обучения.
        """
        self.W -= alpha * self.weights_grad
        self.b -= alpha * self.b_grad
    

class ReLU(Module):
    """
    Слой, соответствующий функции активации ReLU. Данная функция возвращает новый массив, в котором значения меньшие 0 заменены на 0.
    """
    def __init__(self):
        self.input_relu = None
    
    def forward(self, x: np.ndarray) -> np.ndarray:
        """
        Возвращает y = max(0, x).

        Parameters
        ----------
        x : np.ndarray
            Входной вектор или батч.
    
        Return
        ------
        y : np.ndarray
            Выход после слоя (той же размерности, что и вход).

        """
        self.input_relu = x
        return np.maximum(0, x)
        
    def backward(self, d) -> np.ndarray:
        """
        Cчитает градиент при помощи обратного распространения ошибки.

        Parameters
        ----------
        d : np.ndarray
            Градиент.
        Return
        ------
        np.ndarray
            Новое значение градиента.
        """
        relu_grad = self.input_relu > 0
        return d*relu_grad

### Задание 2 (2 балла)
Теперь сделаем саму нейронную сеть.

#### Методы
`fit(X, y)` - обучает нейронную сеть заданное число эпох. В каждой эпохе необходимо использовать [cross-entropy loss](https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html#cross-entropy) для обучения, а так же производить обновления не по одному элементу, а используя батчи.

`predict_proba(X)` - предсказывает вероятности классов для элементов `X`

#### Параметры конструктора
`modules` - список, состоящий из ранее реализованных модулей и описывающий слои нейронной сети. В конец необходимо добавить `Softmax`

`epochs` - количество эпох обучения

`alpha` - скорость обучения

#### Оценка
Оценка производится на заданных ботом гиперпараметрах и архитектурах. Ожидается, что при подобранных заранее гиперпараметрах решение будет демонстрировать приемлемую точность.

Всего 20 тестов по 500 точек в обучающей выборке и по 100 точек в тестовой выборке c 20 эпохами обучения и 10 тестов по 1000 точек в обучающей выборке и 200 точек в тестовой выборке с 40 эпохами обучения. Количество признаков варьируется от 2 до 8. Количество классов не более 8 и не менее 2.

In [3]:
def softmax(x):
    exp_res = np.exp(x - np.max(x))
    return exp_res / np.sum(exp_res, axis = 1, keepdims = True)

def cross_entropy_derivative(y_true, y_pred):
    return y_pred - y_true

class MLPClassifier:
    def __init__(self, modules: List[Module], epochs: int = 40, alpha: float = 0.01, batch_size: int = 32):
        """
        Parameters
        ----------
        modules : List[Module]
            Cписок, состоящий из ранее реализованных модулей и 
            описывающий слои нейронной сети. 
            В конец необходимо добавить Softmax.
        epochs : int
            Количество эпох обучения.
        alpha : float
            Cкорость обучения.
        batch_size : int
            Размер батча, используемый в процессе обучения.
        """
        self.modules = modules
        self.epochs = epochs
        self.alpha = alpha
        self.batch_size = batch_size
            
    def fit(self, X: np.ndarray, y: np.ndarray) -> NoReturn:
        """
        Обучает нейронную сеть заданное число эпох. 
        В каждой эпохе необходимо использовать cross-entropy loss для обучения, 
        а так же производить обновления не по одному элементу, а используя батчи (иначе обучение будет нестабильным и полученные результаты будут плохими.

        Parameters
        ----------
        X : np.ndarray
            Данные для обучения.
        y : np.ndarray
            Вектор меток классов для данных.
        """
        y = np.array(y)
        y_one_hot = np.zeros((y.size, y.max() + 1))
        y_one_hot[np.arange(y.size), y] = 1
        for _ in range(self.epochs):
            new_indices = np.random.permutation(len(y))
            X_shuffled = X[new_indices]
            y_shuffled = y_one_hot[new_indices]
            for k in range(0, len(y_shuffled) - 1, self.batch_size):
                X_batch = X_shuffled[k:k + self.batch_size]
                y_batch = y_shuffled[k:k + self.batch_size]
                output = np.array(X_batch)
                for layer in self.modules:                  
                    output = layer.forward(output)
                output = softmax(output)
                loss_grad = cross_entropy_derivative(y_batch, output)
                for layer in reversed(self.modules):                  
                    loss_grad = layer.backward(loss_grad)
                for layer in self.modules: 
                    layer.update(self.alpha)
                          
        
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Предсказывает вероятности классов для элементов X.

        Parameters
        ----------
        X : np.ndarray
            Данные для предсказания.
        
        Return
        ------
        np.ndarray
            Предсказанные вероятности классов для всех элементов X.
            Размерность (X.shape[0], n_classes)
        
        """
        output = np.array(X)
        for layer in self.modules:                  
            output = layer.forward(output)
        return softmax(output)
        
    def predict(self, X) -> np.ndarray:
        """
        Предсказывает метки классов для элементов X.

        Parameters
        ----------
        X : np.ndarray
            Данные для предсказания.
        
        Return
        ------
        np.ndarray
            Вектор предсказанных классов
        
        """
        p = self.predict_proba(X)
        return np.argmax(p, axis=1)

In [4]:
p = MLPClassifier([
    Linear(4, 8),
    ReLU(),
    Linear(8, 8),
    ReLU(),
    Linear(8, 2)
])

X = np.random.randn(50, 4)
y = np.array([(0 if x[0] > x[2]**2 or x[3]**3 > 0.5 else 1) for x in X])
p.fit(X, y)

### Задание 3 (2 балла)
Протестируем наше решение на синтетических данных. Необходимо подобрать гиперпараметры, при которых качество полученных классификаторов будет достаточным.

Первый датасет - датасет moons. В каждом тесте у данных всего два признака, классов также два.

Второй датасет - датасет blobs. В каждом тесте у данных по два признака, классов три.


Обратите внимание, что датасеты могут отличаться от приведенных ниже по количеству точек, уровню шума и положению центроидов. Количество классов и признаков остается неизменным.

Обратите внимание, что классификатор будет обучаться ботом под каждый датасет отдельно. Обучать самостоятельно в файле `task.py` классификатор не нужно.

Количество датасетов каждого типа равно 5. Количество точек в обучающей выборке не менее 1000, количество точек в тестовой выборке не менее 200.

#### Оценка
Средняя точность на датасетах moons больше 0.85 - +1 балл

Средняя точность на датасетах blobs больше 0.85 - +1 балл

In [5]:
classifier_moons = MLPClassifier([
    Linear(2,4),
    ReLU(),
    Linear(4,6),
    ReLU(),
    Linear(6,2)])
classifier_blobs = MLPClassifier([
    Linear(2, 3),
    ReLU()])


In [6]:
X, y = make_moons(400, noise=0.075)
X_test, y_test = make_moons(400, noise=0.075)
classifier_moons.fit(X, y)
print("Accuracy", np.mean(classifier_moons.predict(X_test) == y_test))

Accuracy 0.88


In [7]:
X, y = make_blobs(400, 2, centers=[[0, 0], [2.5, 2.5], [-2.5, 3]])
X_test, y_test = make_blobs(400, 2, centers=[[0, 0], [2.5, 2.5], [-2.5, 3]])
classifier_blobs.fit(X, y)
print("Accuracy", np.mean(classifier_blobs.predict(X_test) == y_test))

Accuracy 0.9375
